# Train model and do test

In [252]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from preprocess_data import *
from Hyperparameter_optimization import *
from Degree_selection import *
from proj1_helpers import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [250]:
from proj1_helpers import *

## Load the training set
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y_train, tX_train, ids_train = load_csv_data(DATA_TRAIN_PATH)
feature_names = load_headers(DATA_TRAIN_PATH)

# convert y from -1/1 to 0,1
y_train = (y_train+1)/2

# process train set
data_pro  = data_preprocess_train(tX_train,y_train,outlier_method = 'IQR',n_out = 5)

## Load the test set

# Load test set
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
y_test, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
feature_names = load_headers(DATA_TEST_PATH)
# process test set
data_test  = data_preprocess_predict(tX_test,outlier_method = 'IQR',n_out = 5)

# Train a model and see accuracy

In [327]:
#data_pro  = data_preprocess_train(tX_train,y_train,outlier_method = 'IQR',n_out = 5)
#lambda_ = [1e-2,1e-3,1e-4,1e-5]
lambda_ = [0]
#lambda_ = np.linspace(1e-3,1e-5,4)
gamma_ = np.logspace(-4,-1,4) #np.logspace(-15,-14,2)
accuracy_ = []
group_size_ = []
optimal_lambdas = []
degrees = [2, 2, 2, 2, 2, 2]
for idx_data,data in enumerate(data_pro):
    print('┌'+'─' * 38+'┐')
    print('│               Group: %i               │'%idx_data)
    print('└'+'─' * 38+'┘')
    tx = data[0]
  #  tx = np.hstack((build_poly(tx, degrees[idx_data]),np.sin(tx)))
    tx = build_poly(tx, degrees[idx_data])
    y = data[1]. reshape(-1,1)
    initial_w_ = np.zeros((tx.shape[1],1))
    optimal_accuracy, optimal_lambda_, optimal_gamma = Hyperparameter_optimization( y, tx,k_fold=5, 
                                                                                   lambdas=lambda_ , 
                                                                                   gammas = gamma_, 
                                                                                   initial_w = initial_w_, 
                                                                                   max_iters = 100, 
                                                                                   model = 'least_squares_SGD')
                                                                                   #model = 'least_squares')
    accuracy_.append(optimal_accuracy)
    group_size_.append(len(data[2]))
    optimal_lambdas.append(optimal_lambda_)

accuracy_all = np.dot(accuracy_,group_size_)/np.sum(group_size_)
print('\n * Overall Accuracy = %.5f'%accuracy_all)

┌──────────────────────────────────────┐
│               Group: 0               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0,  gamma: 0.0001 
 
 Model least_squares_SGD 
 Loss: 0.28282235045550486 
 Accuracy: 0.6756471066540182
────────────────────────────────────────
lambda : 0,  gamma: 0.001 
 
 Model least_squares_SGD 
 Loss: 0.20470258385707538 
 Accuracy: 0.6728825044043909
────────────────────────────────────────
lambda : 0,  gamma: 0.01 
 
 Model least_squares_SGD 
 Loss: 1.3766668261415258 
 Accuracy: 0.5937931969101504
────────────────────────────────────────
lambda : 0,  gamma: 0.1 
 
 Model least_squares_SGD 
 Loss: 6.916368676217075e+43 
 Accuracy: 0.5414690337444098
════════════════════════════════════════
*The optimal hyperparameters*:
 Model: least_squares_SGD 
 Optimal accuracy: 0.6756471066540182 
 Optimal lambda_: 0 
 Optimal gamma: 0.0001 
 
┌──────────────────────────────────────┐
│               Group: 1            

In [ ]:
optimal_lambdas

# Train model and do prediction on test set

In [10]:
#from preprocess_data import *

#data_pro  = data_preprocess_train(tX_train,y_train,outlier_method = 'IQR',n_out = 5)

# Train
lambda_ = [6.5e-05,
 1e-05,
 0.0009,
 0.000835,
 0.00078,
 0.000615]
loss_ = [] 
w_ = []
for idx_data,data in enumerate(data_pro):
    print('┌'+'─' * 38+'┐')
    print('│               Group: %i               │'%idx_data)
    print('└'+'─' * 38+'┘')
    tx = data[0]
    tx = np.hstack((build_poly(tx, degrees[idx_data]),1/(abs(tx)+1e-10),np.sin(tx)))
    y = data[1]
    initial_w_ = np.ones(tx.shape[1])
    loss_temp, w_temp = ridge_regression(y, tx,lambda_[idx_data])
    w_.append(w_temp)
    print('The training of Group {} is over.'.format(idx_data))
#print('\n * Overall Accuracy = %.4f'%accuracy_all)

print('\nTraining finished! Start do prediction and write to prediction.csv.')

# Using trained model to do prediction
label_ = []
data_size = sum([len(data_test[:,1][i]) for i in range(6)])
y_predict = np.zeros(data_size)
for idx_data,data in enumerate(data_test):
    tx = data[0]
    tx = np.hstack((build_poly(tx, degrees[idx_data]),1/(abs(tx)+1e-10),np.sin(tx)))
    label_.append(predict_labels(w_[idx_data], tx,model='ridge_regression'))
    y_predict[data[1]] = predict_labels(w_[idx_data], tx,model='ridge_regression')

create_csv_submission(range(350000,350000+data_size),(y_predict-0.5)*2,'prediction.csv')

print('\nPrediction finished! Please check corresponding prediction.csv in ../scripts/.')

┌──────────────────────────────────────┐
│               Group: 0               │
└──────────────────────────────────────┘
The training of Group 0 is over.
┌──────────────────────────────────────┐
│               Group: 1               │
└──────────────────────────────────────┘
The training of Group 1 is over.
┌──────────────────────────────────────┐
│               Group: 2               │
└──────────────────────────────────────┘
The training of Group 2 is over.
┌──────────────────────────────────────┐
│               Group: 3               │
└──────────────────────────────────────┘
The training of Group 3 is over.
┌──────────────────────────────────────┐
│               Group: 4               │
└──────────────────────────────────────┘
The training of Group 4 is over.
┌──────────────────────────────────────┐
│               Group: 5               │
└──────────────────────────────────────┘
The training of Group 5 is over.

Training finished! Start do prediction and write to prediction.

In [319]:
x = data_pro[1][0]
#x = np.hstack((build_poly(x, 5),1/(abs(x)+1e-10),np.sin(x)))
#x = np.hstack((build_poly(x, 10),np.sin(x)))
x = build_poly(x, 2)
y = data_pro[1][1].reshape(-1,1)
w = np.zeros((x.shape[1],1))

In [323]:
least_squares_GD(y, x, np.array(w), 1000, 1e-1)

(nan, array([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]]))

In [210]:
w

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [211]:
cross_validation(y, x,5, 0, 1e-10, np.array(w), 20, 'least_squares_GD')

LOSS 0.027099261583267392
LOSS 0.026648334455768946
LOSS 0.026511557983719285
LOSS 0.026447758617281887
LOSS 0.026409782682899178
LOSS 0.02638463178988856
LOSS 0.026367133223390432
LOSS 0.0263545831239723
LOSS 0.026345343151959497
LOSS 0.026338352115607117
LOSS 0.026332902992812653
LOSS 0.026328519017666155
LOSS 0.02632487688059673
LOSS 0.026321756625332406
LOSS 0.0263190082248272
LOSS 0.026316529070828588
LOSS 0.026314248751367834
LOSS 0.026312118737670333
LOSS 0.026310105390698533
LOSS 0.026308185216083343
LOSS 0.027488254466632323
LOSS 0.02697101269067271
LOSS 0.026849195138512822
LOSS 0.0268155821837019
LOSS 0.026803259515594056
LOSS 0.026796690747412616
LOSS 0.02679194988334484
LOSS 0.026787939003141404
LOSS 0.026784314186658477
LOSS 0.02678095272230345
LOSS 0.026777803004872074
LOSS 0.026774838326223616
LOSS 0.026772041513000444
LOSS 0.026769399479463708
LOSS 0.026766901180586132
LOSS 0.026764536793452615
LOSS 0.02676229736248189
LOSS 0.0267601746274723
LOSS 0.026758160927296794


(0.02637837634797502, 0.9432)

In [206]:
w

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],